# LightFm Алгоритм

LightFM — это реализация на Python ряда популярных алгоритмов рекомендаций как для неявной, так и для явной обратной связи.


## Подключение всех используемых библиотек

In [1]:
from pandas.io.json import json_normalize
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder
import numpy as np

from scipy.sparse import coo_matrix
from scipy import sparse
import warnings; warnings.simplefilter('ignore')
import random
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import pickle
import pandas as pd
import time

## Чтение и предобработка данных 

In [2]:
df= pd.read_csv('train_12.csv',index_col=0,low_memory=False)
df

,object.storage.id,object.storage.name,src.ip,subject.account.id,subject.account.domain,event_src.host,src.asset,datafield6,time
0,\\*\w3svc1,u_ex22030722_x.log,10.10.192.57,S-1-5-21-1023191730-727829927-3985050192-32035,ptsecurity.ru,dc1-mail-03.ptsecurity.ru,165dc53b-cd40-0001-0000-0000000007ee,0x120089,2022-03-22T00:45:57Z
1,\\*\w3svc1,u_ex22030723_x.log,10.10.192.57,S-1-5-21-1023191730-727829927-3985050192-32035,ptsecurity.ru,dc1-mail-03.ptsecurity.ru,165dc53b-cd40-0001-0000-0000000007ee,0x120089,2022-03-22T00:45:57Z
2,\\*\w3svc1,u_ex22030806_x.log,10.10.192.57,S-1-5-21-1023191730-727829927-3985050192-32035,ptsecurity.ru,dc1-mail-03.ptsecurity.ru,165dc53b-cd40-0001-0000-0000000007ee,0x120089,2022-03-22T00:45:57Z
3,\\*\w3svc1,u_ex22030809_x.log,10.10.192.57,S-1-5-21-1023191730-727829927-3985050192-32035,ptsecurity.ru,dc1-mail-03.ptsecurity.ru,165dc53b-cd40-0001-0000-0000000007ee,0x120089,2022-03-22T00:45:57Z
4,\\*\w3svc1,u_ex22030810_x.log,10.10.192.57,S-1-5-21-1023191730-727829927-3985050192-32035,ptsecurity.ru,dc1-mail-03.ptsecurity.ru,165dc53b-cd40-0001-0000-0000000007ee,0x120089,2022-03-22T00:45:57Z
...,...,...,...,...,...,...,...,...,...
1919995,\\*\opp,акцепт,10.0.102.29,S-1-5-21-1023191730-727829927-3985050192-18835,ptsecurity.ru,dc2-fs-01.ptsecurity.ru,1457bbaf-9940-0001-0000-000000001019,0x100081,2022-04-07 14:58:07+00:00
1919996,\\*\opp,дуд губернатора самарской области и правительства,10.0.102.29,S-1-5-21-1023191730-727829927-3985050192-18835,ptsecurity.ru,dc2-fs-01.ptsecurity.ru,1457bbaf-9940-0001-0000-000000001019,0x100081,2022-04-07 14:58:07+00:00
1919997,\\*\opp,отгрузка сертификата vm 09.02.22,10.0.102.29,S-1-5-21-1023191730-727829927-3985050192-18835,ptsecurity.ru,dc2-fs-01.ptsecurity.ru,1457bbaf-9940-0001-0000-000000001019,0x100081,2022-04-07 14:58:07+00:00
1919998,\\*\opp,гау ит-парк,10.0.102.29,S-1-5-21-1023191730-727829927-3985050192-18835,ptsecurity.ru,dc2-fs-01.ptsecurity.ru,1457bbaf-9940-0001-0000-000000001019,0x100081,2022-04-07 14:58:07+00:00


In [3]:
# Cортировка столбцов для удобства
df=df[['subject.account.id','object.storage.name',
'object.storage.id',
'src.ip', 'event_src.host','subject.account.domain',
'src.asset','datafield6','time']]

Приведение всех данных столбцов в числовой вид

In [4]:
#Обработка адресов
df=df.loc[(df['src.ip']!='127.0.0.1') & (df['src.ip']!='::1')& (df['src.ip']!='-')]

In [5]:
#обработка времени подключеня пользователя к тому или иному хосту
df['time'] = pd.to_datetime(df['time'])
df["dp"] = pd.to_datetime(df['time'],unit='s').apply(lambda x: "Day" if x.hour >= 10 and x.hour <= 18 else 'Morning'  if x.hour>=6 and x.hour<=10 else 'Evening'  if x.hour>=18 and x.hour<=22 else "Night")
df=df.drop(columns=['time'])

In [6]:
#обработка наименования хранилища, добавление столбца с типом записи 
result=df['object.storage.name'].str.split(r'[^\\]+(?=\.\w+$)',expand=True)
df['ext']=result[1]

In [7]:
df

,subject.account.id,object.storage.name,object.storage.id,src.ip,event_src.host,subject.account.domain,src.asset,datafield6,dp,ext
0,S-1-5-21-1023191730-727829927-3985050192-32035,u_ex22030722_x.log,\\*\w3svc1,10.10.192.57,dc1-mail-03.ptsecurity.ru,ptsecurity.ru,165dc53b-cd40-0001-0000-0000000007ee,0x120089,Night,.log
1,S-1-5-21-1023191730-727829927-3985050192-32035,u_ex22030723_x.log,\\*\w3svc1,10.10.192.57,dc1-mail-03.ptsecurity.ru,ptsecurity.ru,165dc53b-cd40-0001-0000-0000000007ee,0x120089,Night,.log
2,S-1-5-21-1023191730-727829927-3985050192-32035,u_ex22030806_x.log,\\*\w3svc1,10.10.192.57,dc1-mail-03.ptsecurity.ru,ptsecurity.ru,165dc53b-cd40-0001-0000-0000000007ee,0x120089,Night,.log
3,S-1-5-21-1023191730-727829927-3985050192-32035,u_ex22030809_x.log,\\*\w3svc1,10.10.192.57,dc1-mail-03.ptsecurity.ru,ptsecurity.ru,165dc53b-cd40-0001-0000-0000000007ee,0x120089,Night,.log
4,S-1-5-21-1023191730-727829927-3985050192-32035,u_ex22030810_x.log,\\*\w3svc1,10.10.192.57,dc1-mail-03.ptsecurity.ru,ptsecurity.ru,165dc53b-cd40-0001-0000-0000000007ee,0x120089,Night,.log
...,...,...,...,...,...,...,...,...,...,...
1919995,S-1-5-21-1023191730-727829927-3985050192-18835,акцепт,\\*\opp,10.0.102.29,dc2-fs-01.ptsecurity.ru,ptsecurity.ru,1457bbaf-9940-0001-0000-000000001019,0x100081,Day,None
1919996,S-1-5-21-1023191730-727829927-3985050192-18835,дуд губернатора самарской области и правительства,\\*\opp,10.0.102.29,dc2-fs-01.ptsecurity.ru,ptsecurity.ru,1457bbaf-9940-0001-0000-000000001019,0x100081,Day,None
1919997,S-1-5-21-1023191730-727829927-3985050192-18835,отгрузка сертификата vm 09.02.22,\\*\opp,10.0.102.29,dc2-fs-01.ptsecurity.ru,ptsecurity.ru,1457bbaf-9940-0001-0000-000000001019,0x100081,Day,.22
1919998,S-1-5-21-1023191730-727829927-3985050192-18835,гау ит-парк,\\*\opp,10.0.102.29,dc2-fs-01.ptsecurity.ru,ptsecurity.ru,1457bbaf-9940-0001-0000-000000001019,0x100081,Day,None


Обработка пустых данных

In [8]:
obj_df = df.select_dtypes(include=['object']).copy()
obj_df=obj_df.fillna('un')

### Подготовка данных для использования их в алгоритме рекомендательной системы

Создание матрицы отношений Пользователь - Хост

In [9]:
from sklearn.preprocessing import OrdinalEncoder
vectorizer = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)
encoded_df = pd.DataFrame(vectorizer.fit_transform(obj_df))

In [10]:
int_df = df.select_dtypes(include=['float64','int64']).copy()
df_enc=int_df.join(encoded_df)
df_enc=df_enc.fillna(0)

In [11]:
df_enc

,0,1,2,3,4,5,6,7,8,9
0,442.0,24614.0,101.0,450.0,209.0,34.0,1168.0,15.0,2.0,523.0
0,442.0,24614.0,101.0,450.0,209.0,34.0,1168.0,15.0,2.0,523.0
0,442.0,24614.0,101.0,450.0,209.0,34.0,1168.0,15.0,2.0,523.0
0,442.0,24614.0,101.0,450.0,209.0,34.0,1168.0,15.0,2.0,523.0
0,442.0,24614.0,101.0,450.0,209.0,34.0,1168.0,15.0,2.0,523.0
...,...,...,...,...,...,...,...,...,...,...
1919995,442.0,16656.0,61.0,450.0,207.0,34.0,1168.0,45.0,0.0,523.0
1919996,442.0,16670.0,61.0,450.0,207.0,34.0,1168.0,45.0,0.0,523.0
1919997,442.0,16675.0,61.0,450.0,207.0,34.0,1168.0,45.0,0.0,523.0
1919998,442.0,16683.0,61.0,450.0,207.0,34.0,1168.0,45.0,0.0,523.0


In [12]:
df_enc.columns = ['uid','iid','object.storage.id',
'src.ip', 'event_src.host','subject.account.domain',
'src.asset','datafield6','dp','ext']
df_enc

,uid,iid,object.storage.id,src.ip,event_src.host,subject.account.domain,src.asset,datafield6,dp,ext
0,442.0,24614.0,101.0,450.0,209.0,34.0,1168.0,15.0,2.0,523.0
0,442.0,24614.0,101.0,450.0,209.0,34.0,1168.0,15.0,2.0,523.0
0,442.0,24614.0,101.0,450.0,209.0,34.0,1168.0,15.0,2.0,523.0
0,442.0,24614.0,101.0,450.0,209.0,34.0,1168.0,15.0,2.0,523.0
0,442.0,24614.0,101.0,450.0,209.0,34.0,1168.0,15.0,2.0,523.0
...,...,...,...,...,...,...,...,...,...,...
1919995,442.0,16656.0,61.0,450.0,207.0,34.0,1168.0,45.0,0.0,523.0
1919996,442.0,16670.0,61.0,450.0,207.0,34.0,1168.0,45.0,0.0,523.0
1919997,442.0,16675.0,61.0,450.0,207.0,34.0,1168.0,45.0,0.0,523.0
1919998,442.0,16683.0,61.0,450.0,207.0,34.0,1168.0,45.0,0.0,523.0


In [13]:
encoded_df=df_enc[['uid','iid']]

In [14]:
encoded_df.info()
encoded_df['quantity'] = 1
grouped_purchased = encoded_df.groupby(['uid','iid']).sum().reset_index()
grouped_purchased

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2889059 entries, 0 to 1919999
Data columns (total 2 columns):
 #   Column  Dtype  
---  ------  -----  
 0   uid     float64
 1   iid     float64
dtypes: float64(2)
memory usage: 66.1 MB


,uid,iid,quantity
0,0.0,25454.0,11147
1,1.0,25454.0,20
2,2.0,25454.0,148
3,3.0,25454.0,3
4,4.0,25454.0,12
...,...,...,...
61942,1498.0,25454.0,1
61943,1499.0,25454.0,3
61944,1500.0,25454.0,7
61945,1501.0,25454.0,2


## Построение рекомендательной системы на основе LightFM

In [15]:
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm import cross_validation

In [16]:
# Загрузка метрик оценок LightFM
from lightfm.evaluation import precision_at_k as lightfm_prec_at_k
from lightfm.evaluation import recall_at_k as lightfm_recall_at_k

Создание датасета для работы

In [17]:
dataset = Dataset()
dataset.fit(users=encoded_df['uid'], items=encoded_df['iid'])

In [18]:
num_users, num_topics = dataset.interactions_shape()
print(f'Количество пользователей: {num_users}, Количество хостов-переходов: {num_topics}.')

Количество пользователей: 1500, Количество хостов-переходов: 47294.


In [19]:
print_df = pd.DataFrame(
    {'Пользователь': encoded_df['uid'],
     'Хост': encoded_df['iid']
    })

print_df.head()

,Пользователь,Хост
0,442.0,24614.0
0,442.0,24614.0
0,442.0,24614.0
0,442.0,24614.0
0,442.0,24614.0


### Функции по работе с матрицей и настройки модели

In [20]:
def get_user_list(df, user_column):
    
    return np.sort(df[user_column].unique())

def get_item_list(df, item_name_column):
    
    """
    Создание списка элементов из фрейма данных df,
    item_column - это столбец, состоящий из элементов в фрейме данных df
    
    Возвращаемые данные:
    return to item_id_list, item_id, name_mapping    
    """
    
    item_list = df[item_name_column].unique()
    
    
    return item_list

def get_feature_list(aisle_df, department_df, aisle_name_column, department_name_column):
    
    aisle = aisle_df[aisle_name_column]
    department = department_df[department_name_column]
    
    return pd.concat([aisle, department], ignore_index = True).unique()

# creating user_id, item_id, and features_id

def id_mappings(user_list, item_list, feature_list):
    """
    Создание сопоставления идентификаторов для преобразования user_id, item_id и feature_id
    """
    user_to_index_mapping = {}
    index_to_user_mapping = {}
    for user_index, user_id in enumerate(user_list):
        user_to_index_mapping[user_id] = user_index
        index_to_user_mapping[user_index] = user_id
        
    item_to_index_mapping = {}
    index_to_item_mapping = {}
    for item_index, item_id in enumerate(item_list):
        item_to_index_mapping[item_id] = item_index
        index_to_item_mapping[item_index] = item_id
        
    feature_to_index_mapping = {}
    index_to_feature_mapping = {}
    for feature_index, feature_id in enumerate(feature_list):
        feature_to_index_mapping[feature_id] = feature_index
        index_to_feature_mapping[feature_index] = feature_id
        
        
    return user_to_index_mapping, index_to_user_mapping, \
           item_to_index_mapping, index_to_item_mapping, \
           feature_to_index_mapping, index_to_feature_mapping


def get_user_product_interaction(orders_df, order_products_train_df, order_products_test_df, products_df):
    
    #На тренировочных данных
    # создание матрицы данных, состоящий из двух столбцов user_id и item_id
    user_to_product_train_df = orders_df[orders_df["eval_set"] == "prior"][["user_id", "order_id"]].\
    merge(order_products_train_df[["order_id", "product_id"]]).merge(products_df[["product_id", "product_name"]])\
    [["user_id", "product_name"]].copy()
    
    # присвоение рейтинга в виде количества переходов пользователя на тот или иной хост
    user_to_product_train_df["product_count"] = 1
    user_to_product_rating_train = user_to_product_train_df.groupby(["user_id", "product_name"], as_index = False)["product_count"].sum()
    
    #на тестовых данных
    user_to_product_test_df = orders_df[orders_df["eval_set"] == "train"][["user_id", "order_id"]].\
    merge(order_products_test_df[["order_id", "product_id"]]).merge(products_df[["product_id", "product_name"]])\
    [["user_id", "product_name"]].copy()    
    
    user_to_product_test_df["product_count"] = 1
    user_to_product_rating_test = user_to_product_test_df.groupby(["user_id", "product_name"], as_index = False)["product_count"].sum()
    
    # Слияние с предыдущими тренировозными данными user_to_product_rating_training
    
    user_to_product_rating_test = user_to_product_rating_test.\
    merge(user_to_product_rating_train.rename(columns = {"product_count" : "previous_product_count"}), how = "left").fillna(0)
    user_to_product_rating_test["product_count"] = user_to_product_rating_test.apply(lambda x: x["previous_product_count"] + \
                                                                                    x["product_count"], axis = 1)
    user_to_product_rating_test.drop(columns = ["previous_product_count"], inplace = True)
    
    return user_to_product_rating_train, user_to_product_rating_test


def get_interaction_matrix(df, df_column_as_row, df_column_as_col, df_column_as_value, row_indexing_map, 
                          col_indexing_map):
    
    row = df[df_column_as_row].apply(lambda x: row_indexing_map[x]).values
    col = df[df_column_as_col].apply(lambda x: col_indexing_map[x]).values
    value = df[df_column_as_value].values
    
    return coo_matrix((value, (row, col)), shape = (len(row_indexing_map), len(col_indexing_map)))

def get_product_feature_interaction(product_df, aisle_df, department_df, aisle_weight = 1, department_weight = 1):
    item_feature_df = product_df.merge(aisle_df).merge(department_df)[["product_name", "aisle", "department"]]
    
    # Индексирование данных
    item_feature_df["product_name"] = item_feature_df["product_name"]
    item_feature_df["aisle"] = item_feature_df["aisle"]
    item_feature_df["department"] = item_feature_df["department"]    
    
    product_aisle_df = item_feature_df[["product_name", "aisle"]].rename(columns = {"aisle" : "feature"})
    product_aisle_df["feature_count"] = aisle_weight 
    product_department_df = item_feature_df[["product_name", "department"]].rename(columns = {"department" : "feature"})
    product_department_df["feature_count"] = department_weight     
    
    product_feature_df = pd.concat([product_aisle_df, product_department_df], ignore_index=True)
    
    del item_feature_df
    del product_aisle_df
    del product_department_df    
    
    # группировка для суммирования по feature_count
    product_feature_df = product_feature_df.groupby(["product_name", "feature"], as_index = False)["feature_count"].sum()    
    
    return product_feature_df

def get_user_product_interaction(user_to_product):

    user_to_product["product_count"] = 1
    user_to_product_rating_train = user_to_product.groupby(["uid", "iid"], as_index = False)["product_count"].sum()
     
    return user_to_product_rating_train

def id_mappings(user_list, item_list):
    """
    Создание сопоставления идентификаторов для преобразования user_id, item_id и feature_id    
    """
    user_to_index_mapping = {}
    index_to_user_mapping = {}
    for user_index, user_id in enumerate(user_list):
        user_to_index_mapping[user_id] = user_index
        index_to_user_mapping[user_index] = user_id
        
    item_to_index_mapping = {}
    index_to_item_mapping = {}
    for item_index, item_id in enumerate(item_list):
        item_to_index_mapping[item_id] = item_index
        index_to_item_mapping[item_index] = item_id
        
    return user_to_index_mapping, index_to_user_mapping, \
           item_to_index_mapping, index_to_item_mapping, \

#=======================
    # converting to coo_matrix
    
    row = product_feature_df["product_name"].values,
    col = product_feature_df["feature"].values,
    value = product_feature_df["feature_count"].values
    return coo_matrix((value, (row, col)))
    

Создание списков

In [21]:
users = get_user_list(encoded_df, "uid")
items = get_item_list(encoded_df, "iid")
features = get_feature_list(df, df, "object.storage.id", "datafield6")

Пользователи

In [22]:
users

array([0.000e+00, 1.000e+00, 2.000e+00, ..., 1.500e+03, 1.501e+03,
       1.502e+03])

Объекты пользователя- хосты

In [23]:
items

array([24614., 24615., 24624., ...,  3896., 56425., 54739.])

In [24]:
user_to_index_mapping, index_to_user_mapping, \
           item_to_index_mapping, index_to_item_mapping = id_mappings(users, items)

In [39]:
user_to_product_rating_train = get_user_product_interaction(encoded_df)

In [26]:
# короче вот где весь прикол: строки и столбцы матрицы - это индексы продуктов и юзеров. Индексы в дф, Карл,
# только тогда эта библиотека понимает и нормально работает.
def get_interaction_matrix(df, df_column_as_row, df_column_as_col, df_column_as_value, row_indexing_map,
                          col_indexing_map):
    
    row = df[df_column_as_row].apply(lambda x: row_indexing_map[x]).values
    col = df[df_column_as_col].apply(lambda x: col_indexing_map[x]).values
    value = df[df_column_as_value].values
    
    return coo_matrix((value, (row, col)), shape = (len(row_indexing_map), len(col_indexing_map)))

In [27]:
user_to_product_interaction_train = get_interaction_matrix(user_to_product_rating_train, "uid", 
                                                    "iid", "product_count", user_to_index_mapping, item_to_index_mapping)

## Обучение модели для рекомендательной системы

In [57]:
model_without_features = LightFM(loss = "warp")
start = time.time()
model_without_features.fit(user_to_product_interaction_train,
          user_features=None, 
          item_features=None, 
          sample_weight=None, 
          epochs=10, 
          num_threads=4,
          verbose=1)

end = time.time()
print("Потраченное на обучение время = {0:.{1}f} секунд".format(end - start, 2))

Epoch: 100%|██████████| 10/10 [00:01<00:00,  6.39it/s]

Потраченное на обучение время = 1.57 секунд


In [46]:
class recommendation_sampling:
    
    def __init__(self, model, items = items, user_to_product_interaction_matrix = user_to_product_interaction_train, 
                user2index_map = user_to_index_mapping):
        
        self.user_to_product_interaction_matrix = user_to_product_interaction_matrix
        self.model = model
        self.items = items
        self.user2index_map = user2index_map
    
    def recommendation_for_user(self, user):
        
        # получение индексов пользователя
        
        userindex = self.user2index_map.get(user, None)
        
        if userindex == None:
            return None
        
        users = userindex
        
        # хосты, которые уже были посещены
        
        known_positives = self.items[self.user_to_product_interaction_matrix.tocsr()[userindex].indices]
        
        scores = self.model.predict(user_ids = users, item_ids = np.arange(self.user_to_product_interaction_matrix.shape[1]))
        
        # топ переходов
        
        top_items = self.items[np.argsort(-scores)]
        
        # Вывод результатов
        print("Пользователь %s" % user)
        print("     Уже известные посещаемые хосты:")
        
        for x in known_positives[:10]:
            print("                  %s" % x)
            
            
        print("     Рекомендации:")
        
        for x in top_items[:10]:
            print("                  %s" % x)

    def recommendation_for_many(self, users):

      predictions = []

      for i in users:
        user = self.user2index_map.get(i)
        scores = self.model.predict(user_ids = int(user), item_ids = np.arange(self.user_to_product_interaction_matrix.shape[1]))
        top_items = self.items[np.argsort(-scores)]
        predictions.append(top_items[:10].tolist())
      
      return predictions 

In [39]:
recom = recommendation_sampling(model = model_without_features)

## Рекомендации

Рекомендация по конкретному пользователю

In [48]:
#например по случайному 10 в списке
recom.recommendation_for_user(10)

Пользователи 10
     Известные позитивные хосты-переходы:
                  22057.0
                  7054.0
                  7996.0
                  11933.0
                  21662.0
                  463.0
                  2426.0
                  25454.0
                  553.0
                  467.0
     Рекомендации:
                  7054.0
                  11933.0
                  22057.0
                  25454.0
                  22123.0
                  7996.0
                  48796.0
                  2426.0
                  5978.0
                  35172.0


In [49]:
recom.recommendation_for_user(3)

Пользователи 3
     Известные позитивные хосты-переходы:
                  25454.0
     Рекомендации:
                  7054.0
                  25454.0
                  22057.0
                  11933.0
                  7996.0
                  22123.0
                  2426.0
                  2326.0
                  2217.0
                  35172.0


In [51]:
#предсказание будущих переходов
predicts = recom.recommendation_for_many(encoded_df['uid'].unique())

In [52]:
predicts

[[24407.0,
  15904.0,
  23629.0,
  22896.0,
  16260.0,
  23041.0,
  15952.0,
  23222.0,
  25043.0,
  6075.0],
 [7054.0,
  25454.0,
  22057.0,
  11933.0,
  7996.0,
  22123.0,
  2426.0,
  2326.0,
  2217.0,
  35172.0],
 [7054.0,
  25454.0,
  11933.0,
  22057.0,
  7996.0,
  22123.0,
  2426.0,
  13708.0,
  14135.0,
  35172.0],
 [22057.0,
  25454.0,
  11933.0,
  7054.0,
  2426.0,
  7996.0,
  2326.0,
  2217.0,
  22123.0,
  35172.0],
 [7054.0,
  25454.0,
  22057.0,
  14485.0,
  14629.0,
  11933.0,
  9280.0,
  22123.0,
  7996.0,
  154.0],
 [7054.0,
  25454.0,
  22057.0,
  11933.0,
  7996.0,
  22123.0,
  2426.0,
  2217.0,
  2326.0,
  14629.0],
 [7054.0,
  154.0,
  25454.0,
  14629.0,
  9280.0,
  14485.0,
  22057.0,
  11933.0,
  22123.0,
  7996.0],
 [7054.0,
  25454.0,
  22057.0,
  11933.0,
  7996.0,
  22123.0,
  14629.0,
  5402.0,
  21662.0,
  7499.0],
 [25454.0,
  7054.0,
  22057.0,
  11933.0,
  7996.0,
  22123.0,
  7499.0,
  14629.0,
  5402.0,
  154.0],
 [7054.0,
  25454.0,
  22057.0,
  11933.

In [41]:
from pyod.models.copod import COPOD
start = time.time()
clf = COPOD()
clf.fit(user_to_product_interaction_train.toarray())
end = time.time()
print("Потраченное на обучение время = {0:.{1}f} секунд".format(end - start, 2))

Потраченное на обучение время = 18.30 секунд


In [ ]:
y_train_pred = clf.predict(user_to_product_interaction_train) # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

In [46]:
y_train_pred

array([0, 0, 0, ..., 0, 0, 0])

In [47]:
y_train_scores

array([26.64841794, 21.14988452, 23.02853136, ..., 21.20737954,
       21.36577424, 21.41095688])

In [52]:
def interactions(data, row, col, value, row_map, col_map):
    
    #converting the row with its given mappings
    row = data[row].apply(lambda x: row_map[x]).values
    #converting the col with its given mappings
    col = data[col].apply(lambda x: col_map[x]).values
    value = data[value].values
    #returning the interaction matrix
    return coo_matrix((value, (row, col)), shape = (len(row_map), len(col_map)))

In [56]:
from sklearn.model_selection import train_test_split
user_to_product_train,user_to_product_test = train_test_split(user_to_product,test_size=0.33, random_state=42)

NameError: name 'user_to_product' is not defined

In [54]:
user_to_product_interaction_test = interactions(user_to_product_test_df, "CustomerID","Product Name", "Quantity", user_to_index_mapping,
                                                item_to_index_mapping)


NameError: name 'user_to_product_test_df' is not defined